In [1]:
import os
import pybedtools

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

from scipy.stats import mannwhitneyu

In [2]:
os.getcwd()

'/NFS4/storage/yz2676/data/STARR-seq/src'

In [3]:
DESIGN_DIR="../output/Junke_architecture/deep-ATAC-STARR_5"
FULL="distal.full"
PAUSE_DOWN="distal.pause.down"
PAUSE_UP="distal.pause.up"
TATA_DOWN="distal.tata.down"
TATA_UP="distal.tata.up"

DESEQ_DIR=f"{DESIGN_DIR}/DESeq2/"

### Presence of general motifs in TREs

In [4]:
full_lfc = pd.read_csv(f"{DESEQ_DIR}/{FULL}_lfc.txt", sep="\t", header=0)
pause_down_lfc = pd.read_csv(f"{DESEQ_DIR}/{PAUSE_DOWN}_lfc.txt", sep="\t", header=0)
pause_up_lfc = pd.read_csv(f"{DESEQ_DIR}/{PAUSE_UP}_lfc.txt", sep="\t", header=0)
tata_down_lfc = pd.read_csv(f"{DESEQ_DIR}/{TATA_DOWN}_lfc.txt", sep="\t", header=0)
tata_up_lfc = pd.read_csv(f"{DESEQ_DIR}/{TATA_UP}_lfc.txt", sep="\t", header=0)

In [5]:
### Get the corresponding full and partial elements
full_pause_lfc = full_lfc[(full_lfc["enhancer"]==True) &
                          ((full_lfc["name"].isin(pause_down_lfc["name"])) |
                          (full_lfc["name"].isin(pause_up_lfc["name"])))]
pause_down_lfc = pause_down_lfc[pause_down_lfc["name"].isin(full_pause_lfc["name"])]
pause_up_lfc = pause_up_lfc[pause_up_lfc["name"].isin(full_pause_lfc["name"])]
pause_lfc = pd.concat([full_pause_lfc, pause_down_lfc, pause_up_lfc])

full_tata_lfc = full_lfc[(full_lfc["enhancer"]==True) &
                            ((full_lfc["name"].isin(tata_down_lfc["name"])) |
                            (full_lfc["name"].isin(tata_up_lfc["name"])))]
tata_down_lfc = tata_down_lfc[tata_down_lfc["name"].isin(full_tata_lfc["name"])]
tata_up_lfc = tata_up_lfc[tata_up_lfc["name"].isin(full_tata_lfc["name"])]
tata_lfc = pd.concat([full_tata_lfc, tata_down_lfc, tata_up_lfc])

In [6]:
def calculate_mean_decrease_btw_groups(df, group_col):
    output = {}
    full = df[df[group_col] == "Others"].reset_index(drop=True)
    partial = df[df[group_col] != "Others"].reset_index(drop=True)

    for _, row in partial.iterrows():
        full_index = row["name"]
        group = row[group_col]
        activity = row["log2FC"] - full[full["name"] == full_index]["log2FC"].values[0]
        
        if group not in output:
            output[group] = [activity]
        else:
            output[group].append(activity)

    return output

In [7]:
tata = calculate_mean_decrease_btw_groups(tata_lfc, "Groups")
pause = calculate_mean_decrease_btw_groups(pause_lfc, "Groups")

In [8]:
no_tss = [item for k, sublist in tata.items() if k != "StrongTATA" for item in sublist]
have_tss = tata["StrongTATA"]

n_have = len(have_tss)
n_no = len(no_tss)

colors = ["#DCE2E5", "#FFD6E8"]

plt.figure(figsize=(6,5))
sns.boxplot([no_tss, have_tss],width=0.6,patch_artist=True, showfliers=False,
            palette=colors,medianprops=dict(color="black"))

ax = plt.gca()  # Get current axes
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.ylabel("Change in enhancer activity (\u0394log2FC)", fontsize=14)
plt.xticks([0,1], ['Inactive TATA\n(n='+str(n_no)+")", 'Active TATA\n(n='+str(n_have)+")"], fontsize=12)
plt.grid(False)
tata_out_path = f"{DESIGN_DIR}/Visualization/tata_presence.pdf"
plt.savefig(tata_out_path, bbox_inches='tight', dpi=300)
plt.close()
p_value_tss = stats.ranksums(have_tss, no_tss)
p_value_tss

RanksumsResult(statistic=1.021376461713902, pvalue=0.30707611289773684)

In [9]:
no = [item for k, sublist in pause.items() if k != "StrongDPR" for item in sublist]
good = pause["StrongDPR"]

n_good = len(good)
n_no = len(no)

colors = ["#DCE2E5", "#CCC7FF"]

plt.figure(figsize=(6,5))
sns.boxplot([no, good],width=0.6,patch_artist=True, showfliers=False, palette=colors,
            medianprops=dict(color="black"))

ax = plt.gca()  # Get current axes
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.ylabel("Change in enhancer activity (\u0394log2FC)", fontsize=14)
plt.xticks([0, 1], ['Inactive DPR\n(n='+str(n_no)+")", 'Active DPR\n(n='+str(n_good)+")"], fontsize=12)
plt.grid(False)
dpr_out_path = f"{DESIGN_DIR}/Visualization/dpr_presence.pdf"
plt.savefig(dpr_out_path, bbox_inches='tight', dpi=300)
plt.close()
p_value_ps = stats.ranksums(good, no)
p_value_ps

RanksumsResult(statistic=-0.9357545396969165, pvalue=0.3493995825648861)

### Compare to negative controls (STARR-active regions)

In [10]:
average_full_pause_length = (full_pause_lfc["end"] - full_pause_lfc["start"]).mean()
partial_pause = pd.concat([pause_down_lfc, pause_up_lfc])
average_pause_length = (partial_pause["end"] - partial_pause["start"]).mean()

average_full_tata_length = (full_tata_lfc["end"] - full_tata_lfc["start"]).mean()
average_full_tata_length
partial_tata = pd.concat([tata_down_lfc, tata_up_lfc])
average_tata_length = (partial_tata["end"] - partial_tata["start"]).mean()

print(f"Full pause length: {average_full_pause_length}\n\
Partial pause length: {average_pause_length}\n\
Full tata length: {average_full_tata_length}\n\
Partial tata length: {average_tata_length}")

Full pause length: 312.13793103448273
Partial pause length: 245.6
Full tata length: 351.72
Partial tata length: 235.77142857142857


In [ ]:
DNA_path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/deep_ATAC_STARR/processing_data_v1/out_DNA_no_UMI/"
RNA_path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/deep_ATAC_STARR/processing_data_v1/out_RNA_with_UMI/"
dnas = 6
rnas = 4
dna_umi = False
rna_umi = True

In [ ]:
file_list = []
for dna_idx in range(1, dnas + 1):
    dna_raw_count_path = DNA_path + f"DNA{dna_idx}/all/binned_frag_count_binSize_50.bed"
    file_list.append(dna_raw_count_path)
    
for rna_idx in range(1, rnas + 1):
    rna_paths = {
        1: "RNA1/all/",
        2: "corrected_bam/RNA1/all/",
        3: "RNA3/all/",
        4: "corrected_bam_RNA4/RNA1/all/"
    }
    rna_raw_count_path = RNA_path + rna_paths.get(rna_idx) + "binned_frag_count_binSize_50.bed"
    file_list.append(rna_raw_count_path)

In [ ]:
from functools import reduce

processed_dfs = []
for idx, filepath in enumerate(file_list):
    print(f"Reading file: {filepath}")
    file = pybedtools.BedTool(filepath).to_dataframe(disable_auto_names=True, header=None)
    file = file.rename(columns={4: f"value_{idx}"})
    processed_dfs.append(file)

keep_cols = [0, 1, 2, 3]

def merge_strategy(left, right):
    return pd.merge(
        left,
        right,
        on=keep_cols,
        how='outer'
    ).fillna(0)

In [ ]:
combined_df = reduce(
    merge_strategy,
    processed_dfs,
    pd.DataFrame(columns=keep_cols)
)

BINNED_DIR = "/fs/cbsuhy02/storage/yz2676/STARR/data/binned_regions"
combined_df.to_csv(f"{BINNED_DIR}/count_mat_sorted.bed", sep="\t", header=False, index=False, compression="gzip")

In [11]:
annotated_deseq_file_path = "/fs/cbsuhy02/storage/yz2676/STARR/data/DESeq/deep_ATAC_STARR/ori_ind/DE_results_annotated.txt"
atac_peak = "/fs/cbsuhy01/storage/jz855/Reference/K562/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz"

In [12]:
file = pd.read_csv(annotated_deseq_file_path, sep="\t", header=None)

file["Length_groups"] = (file[2]-file[1]).astype(int)

file[1] = file[1].astype(int)
file[2] = file[2].astype(int)
print(len(file))
starr = pybedtools.BedTool.from_dataframe(file.iloc[:,:3])
atac_peak = pybedtools.BedTool(atac_peak)

overlapped_with_ATAC_peak = starr.intersect(atac_peak, f=0.90).to_dataframe().drop_duplicates()
print(len(overlapped_with_ATAC_peak))

starr_overlapped_with_ATAC_peak = file.rename(columns={0:"chrom", 1:"start", 2:"end", 3: "strand", 4: "activity"}).merge(overlapped_with_ATAC_peak, on=["chrom", "start", "end"], how="inner")
print(len(starr_overlapped_with_ATAC_peak))

2407982
1613692
2139647


In [13]:
starr_overlapped_with_ATAC_peak

,chrom,start,end,strand,activity,Length_groups
0,chr1,10051,10201,-,0.315901,150
1,chr1,17251,17601,-,-1.751916,350
2,chr1,17301,17501,-,-1.811366,200
3,chr1,17351,17501,+,-4.266583,150
4,chr1,17401,17551,-,-4.001633,150
...,...,...,...,...,...,...
2139642,chrX,156030351,156030701,-,-5.162984,350
2139643,chrY,11215101,11215351,-,-4.461352,250
2139644,chrY,11215101,11215401,+,-4.484973,300
2139645,chrY,11215151,11215351,-,-3.418095,200


In [14]:
pause_nctrl = starr_overlapped_with_ATAC_peak[starr_overlapped_with_ATAC_peak["Length_groups"].isin([250,300])]
tata_nctrl = starr_overlapped_with_ATAC_peak[starr_overlapped_with_ATAC_peak["Length_groups"].isin([250,350])]

In [15]:
pause_nctrl["Group"] = "Pause Region"
tata_nctrl["Group"] = "Entire Core Promoter"
pause_nctrl["Category"] = pause_nctrl["Length_groups"].map({250: "Partial", 300: "Full"})
tata_nctrl["Category"] = tata_nctrl["Length_groups"].map({250: "Partial", 350: "Full"})

nctrl = pd.concat([pause_nctrl, tata_nctrl], ignore_index=True)

/tmp/ipykernel_3589739/1679405530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pause_nctrl["Group"] = "Pause Region"
/tmp/ipykernel_3589739/1679405530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tata_nctrl["Group"] = "Entire Core Promoter"
/tmp/ipykernel_3589739/1679405530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [16]:
# Define group-specific color palettes
group_palettes = {
    "Pause Region": {"Full": '#B6ADFF', "Partial": '#DAD6FF'},
    "Entire Core Promoter": {"Full": '#FFADD1', "Partial": '#FFD6E8'}  # Example Tata colors
}

groups = nctrl['Group'].unique()
category_order = ['Full', 'Partial']

for group in groups:
    plt.figure(figsize=(6, 8))
    df_group = nctrl[nctrl['Group'] == group].copy()

    df_group["Category"] = pd.Categorical(
        df_group["Category"], 
        categories=category_order,  # Use predefined order
        ordered=True
    )
    df_group = df_group.sort_values("Category")  # Physical sort
    
    # Create violin plot with group-specific colors
    ax = sns.violinplot(
        x="Category", 
        y="activity", 
        hue="Category",
        data=df_group,
        palette=[group_palettes[group][cat] for cat in category_order],
        hue_order=category_order,
        linewidth=1.2,
        order=category_order,
        dodge=False,
        legend=False
    )
    
    # Calculate average lengths (rest of code remains the same)
    avg_lengths = df_group.groupby('Category')['Length_groups'].mean().round(0).astype(int)
    
    new_labels = [
        f"{cat}\n(Avg Length={avg_lengths[cat]}bp)"
        for cat in category_order
    ]
    
    ax.set_xticklabels(new_labels, 
                      fontsize=14, 
                      rotation=0, 
                      ha='center')

    # Statistical test code remains unchanged
    full_data = df_group[df_group['Category'] == 'Full']['activity']
    partial_data = df_group[df_group['Category'] == 'Partial']['activity']
    
    if not full_data.empty and not partial_data.empty:
        stat, p_value = mannwhitneyu(full_data, partial_data, alternative='greater')
        p_text = ('****' if p_value < 0.0001 else
                  '***' if p_value < 0.001 else
                  '**' if p_value < 0.01 else
                  '*' if p_value < 0.05 else 'n.s.')
        
        y_max = df_group['activity'].max() + 1.5
        ax.plot([0, 1], [y_max, y_max], lw=1.5, color='black')
        ax.text(0.5, y_max + 0.25, p_text, 
                ha='center', va='bottom', fontsize=14, color='black')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.title(group, fontsize=20, pad=20)
    plt.ylabel('Activity (log2)', fontsize=17)
    plt.yticks(fontsize=14)
    plt.xlabel('')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95]) 
    
    save_group = group.replace(' ', '_')
    plt.savefig(
        f"{DESIGN_DIR}/Visualization/nctrl_{save_group}.pdf",
        bbox_inches='tight',
        dpi=300
    )
    plt.close()

/tmp/ipykernel_3589739/138296360.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_lengths = df_group.groupby('Category')['Length_groups'].mean().round(0).astype(int)
/tmp/ipykernel_3589739/138296360.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(new_labels,
/tmp/ipykernel_3589739/138296360.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_lengths = df_group.groupby('Category')['Length_groups'].mean().round(0).astype(int)
/tmp/ipykernel_3589739/138296360.py:43: UserWarning: set_ticklabels